In [ ]:
#code to create price bracket
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("PAD.Long.2021.csv")
l = ["R1", "R2", "R3", "R4"]
df = df[df["FY2021.LU"].isin(l)]
df["Monthly Rent"] = df.apply(lambda row: row["FY2021.AV"] * 0.008, axis = 1)
df[["Monthly Rent", "FY2021.AV"]]

df["price bracket"] = df['FY2021.AV'].apply(lambda x: round(x, -4))
df["price bracket"] = df["price bracket"].apply(
    lambda x: '$1000000+' if len(str(x)) >= 9
    else '$' + str(int((round(x, -5)))) + "-" + str(int(round(x, -5) + 100000)) if round(x, -5) <= (round(x, -4))
    else '$' + str(int(round(x, -5) - 100000)) + "-" + str(int(round(x, -5))) if round(x, -5) > (round(x, -4))
    else None
)
df2 = df.groupby("price bracket")["Monthly Rent"].mean()
l2 = []
for i in range(len(df2)):
    df2[i] = int(round(df2[i]))
    l2.append(df2[i])
#l3 = ["0 - 100000", "100000-200000", "200000-300000", "300000-400000", "400000-500000", "500000-600000", "600000-700000", "700000-800000", "800000-900000", "900000-1000000", "1000000+"]
df=df.dropna(subset=["Monthly Rent"])

In [ ]:
#code to create AMI destribution
df["Annual Rent"] = df.apply(lambda row: int(row["Monthly Rent"] * 12), axis = 1)
df["Expected Salary"] = df.apply(lambda row: int(row["Annual Rent"] / 0.3), axis = 1)

sal = [28450, 47400, 56880, 63450, 72550, 90650, 108800, 122400]
ami = ["30%", "50%", "60%", "70%", "80%", "100%", "120%", "130%+"]
temp = []
for k in df["Expected Salary"]:
    bol = False
    for i in range(len(sal) - 1):
        mid = int((sal[i] + sal[i+1])//2)
        if int(k) <= mid:
            temp.append(i)
            bol = True
            break
    if bol == False:
        temp.append(i + 1)

df["label"] = temp
df["AMI"] = df.apply(lambda row: ami[row["label"]], axis = 1)

some = df.groupby(by=["AMI"], sort=True)["FY2021.AV"].mean()
some = some.sort_values(ascending=True)
some2 = []
for k in some:
    some2.append(int(round(k)))

plt.bar(ami, some2)
plt.xlabel('AMI')
plt.ylabel('Average Property Cost')
plt.show()

In [ ]:
#code to classify if listing is affordable or not by comparing to fair rent price
df.loc[df["FY2021.LU"] == "R1", "Family Number"] = 1
df.loc[df["FY2021.LU"] == "R2", "Family Number"] = 2
df.loc[df["FY2021.LU"] == "R3", "Family Number"] = 3
df.loc[df["FY2021.LU"] == "R4", "Family Number"] = 4

df["Rent_Per_Family"] = df.apply(lambda row: row["Monthly Rent"] // row["Family Number"], axis = 1)
df[["Monthly Rent", "Family Number", "Rent_Per_Family"]]

df_income = pd.read_csv("Income Limits (September 8, 2022) - Small Area FMRs.csv")
df_income = df_income.rename(columns={'2023 Boston Small Area Fair Market Rents, Effective 10/1/2022 (https://www.huduser.gov/portal/datasets/fmr/smallarea/index.html)': "ZIP"})
df_income = df_income.rename(columns={'Unnamed: 6': "3bed"})
df_income = df_income.drop(0)
df_income = df_income.drop(31)
zip_list = []

dic = dict(zip(df_income.ZIP, df_income["3bed"]))
for key in dic:
    elem = dic[key]
    dic[key] = elem[1] + elem[3:]

l = []
some = 0
mean_val = 0
for val in dic.values():
    mean_val += float(val)
mean_val = mean_val // len(dic)
for k, y  in zip(df.ZIPCODE, df.Rent_Per_Family):
    x = str(k)
    k = '0' + x[0:-2]
    if k in dic and y < float(dic[k]):
        l.append("Yes")
    elif k in dic:
        l.append("No")
    elif y < mean_val:
        l.append("Yes")
    else:
        l.append("No")

df["affordable"] = l

In [ ]:
#code to match listing with violation dataset
import numpy as np

df_vio = pd.read_csv("Violations2010_2021.csv", encoding='cp1252')
df_vio
test = df[["X", "Y"]]
test2 = df_vio[["longitude", "latitude"]]
test2 = test2.rename(columns={'longitude': 'X', 'latitude': 'Y'})

df = pd.merge(df, test2, on=['X','Y'], how='left', indicator='Violation')
df['Violation'] = np.where(df.Violation == 'both', 1, 0)
df = df.drop_duplicates(keep='first')
df

In [ ]:
#code to calculate percent of afforable vs non-affordable listing with violations
x = df.groupby(["ZIPCODE", "affordable"])["Violation"].sum()
temp = df["ZIPCODE"].unique()
zip_listed = sorted(temp)
vio_afford = [0, 10, 1, 19, 2, 1, 2, 0, 26, 7, 36, 41, 78, 53, 21, 28, 28, 15, 17, 17, 8, 4, 21, 8, 3]
vio_nonafford = [6, 1, 0, 3, 15, 4, 17, 10, 6, 5, 6, 10, 17, 27, 0, 22, 7, 15, 17, 10, 12, 14, 16, 9, 0]
zip_listed.remove(2026.0)
zip_listed.remove(2199.0)
zip_listed.remove(2445.0)
zip_listed.remove(2446.0)
zip_listed.remove(2467.0)

num_afford = df.groupby(["ZIPCODE", "affordable"])["affordable"].count()
num_affordl = list(num_afford)
num_affordl = num_affordl[1:-6]
num_affordl.pop(-3)
len(num_affordl)
afford = []
nonafford = []
for k in range(len(num_affordl)):
    if k % 2 == 0:
      nonafford.append(num_affordl[k])  
    else:
        afford.append(num_affordl[k])

for i in range(25):
    vio_afford[i] = (vio_afford[i] / afford[i]) * 100
    vio_nonafford[i] = (vio_nonafford[i] / nonafford[i]) * 100

In [ ]:
#plug the graph
index = np.arange(25)
bar_width = 0.4

fig, ax = plt.subplots()
af = ax.bar(index, vio_afford, bar_width,
                label="Affordable")

nonaf = ax.bar(index+bar_width, vio_nonafford,
                 bar_width, label="Non-affordable")

ax.set_xlabel('Zip Code')
ax.set_ylabel('Percent of affordable and non-afforadble violations')
ax.set_title('Affordability Influence on Number of Violations')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(zip_listed)
ax.legend()

fig.set_figwidth(20)
fig.set_figheight(10)

plt.show()